# Chương 9: Không gian vector (I)
File enwiki-20150112-400-r10-105752.txt.bz2 là file nén dạng bzip2 của 105752 file text được lấy mẫu ngẫu nhiên (tỷ lệ 1/10) từ các bài báo trên Wikipedia có trên 400 từ. Các bài báo trên Wikipedia được lấy vào ngày 12 tháng 1 năm 2015. Sử dụng dữ liệu file này làm corpus để học các vector thể hiện ý nghĩa của các từ. Trong nửa đầu của chương 9, bạn được yêu cầu trích xuất các context của các từ, trích xuất đặc trưng, và dùng phương pháp PCA để giảm bớt số chiều của dữ liệu. Nửa sau của chương 9 yêu cầu bạn tính độ tương tự của các từ sử dụng các word vectors đã học từ corpus.

Chú ý, bài 83 yêu cầu 7GB memory. Trong trường hợp lượng memory của bạn không đủ, bạn cần có các phương pháp xử lý thích hợp hoặc sử dụng sample 1/100 của dữ liệu trong file enwiki-20150112-400-r10-105752.txt.bz2.

# Download data

In [4]:
!wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2

--2020-04-18 15:55:04--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221903910 (212M) [application/x-bzip2]
Saving to: ‘enwiki-20150112-400-r10-105752.txt.bz2’

enwiki-20150112-400 100%[===================>] 211.62M  11.2MB/s    in 19s     

2020-04-18 15:55:24 (11.1 MB/s) - ‘enwiki-20150112-400-r10-105752.txt.bz2’ saved [221903910/221903910]



In [0]:
!bzip2 -d enwiki-20150112-400-r10-105752.txt.bz2

# 80. Tiền xử lý dữ liệu
Sử dụng khoảng trắng là ký tự ngăn cách để tokenize các từ trong các câu. Phương pháp này có nhược điểm là các từ thu được sẽ còn các ký tự đặc biệt như dấu câu, hoặc dấu ngoặc. Vì thế sau khi tokenize các từ trong corpus, tiến hành các xử lý sau đây.

* Xoá các ký tự đặc biệt xuất hiện ở đầu và cuối các từ: .,!?;:()[]'"
* Xoá các từ chỉ gồm ký tự trắng

Sau khi tiền xử lý dữ liệu, lưu file dữ liệu gồm danh sách các từ cách nhau bởi khoảng trắng.


In [6]:
!head enwiki-20150112-400-r10-105752.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.
As a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divided int

In [7]:
with open('enwiki-20150112-400-r10-105752.txt', 'r') as f:
    lines = f.readlines()

len(lines)

2875326

In [0]:
import re

with open('enwiki_tokenized.txt', 'w') as f:
    for line in lines:
        line = line.strip()

        if line == '':
            continue

        # Remove special characters
        line = re.sub(r'[\.,!\?;:\(\)\[\]\'\"]', '', line)

        # Split by space
        line = line.split(' ')

        # Remove all empty elements
        line = list(filter(lambda a: a != '', line)) # Maybe use filter is faster than other methods

        # Join list by space
        line = ' '.join(line)

        # Write to file
        f.write(line + '\n')

In [23]:
!head 'enwiki_tokenized.txt'

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

In [24]:
# Count number of lines (Linux)
!wc -l 'enwiki_tokenized.txt'

2557968 enwiki_tokenized.txt


In [25]:
# Count number of words (Linux)
!wc -w 'enwiki_tokenized.txt'

120307584 enwiki_tokenized.txt


# 81. Xử lý tên các nước tạo thành từ các compound words
Trong tiếng Anh, nhiều từ cạnh nhau có thể tạo thành một từ có ý nghĩa. Ví dụ, hợp chủng quốc Hoa Kỳ là "United States", vương quốc Anh là "United Kingdom". Nếu chỉ dùng các "United", "State", hay "Kingdom" như các từ riêng lẻ, ý nghĩa của các từ này sẽ nhập nhằng. Vì thế trong khi tiền xử lý dữ liệu, ta cần xác định các từ ghép này. Đoán nhận các từ ghép là một bài toán khó, nên ở đây ta chỉ đoán nhận các từ ghép là tên của các nước.

Trước hết, download danh sách tên của các nước trên Internet. Dùng danh sách tên các nước này để đoán nhận các từ ghép là tên nước trong dữ liệu sử dụng ở bài 80, sau đó biến đổi các ký tự spaces thành ký tự underscore (_) để nối các từ thành phần. Ví dụ "United States" sẽ trở thành "United_States", "Isle of Man" sẽ trở thành "Isle_of_Man."

In [37]:
# Download list country names from github
!wget -O country_names.txt https://raw.githubusercontent.com/datasets/country-list/master/data.csv

--2020-04-18 16:58:46--  https://raw.githubusercontent.com/datasets/country-list/master/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3870 (3.8K) [text/plain]
Saving to: ‘country_names.txt’

country_names.txt   100%[===================>]   3.78K  --.-KB/s    in 0s      

2020-04-18 16:58:46 (13.0 MB/s) - ‘country_names.txt’ saved [3870/3870]



In [0]:
# Get names only
import pandas as pd

country_name_df = pd.read_csv('country_names.txt')

country_name_df['Name'].to_csv('country_names.txt', index=False, header=False)

In [39]:
!head country_names.txt

Afghanistan
Åland Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda


In [0]:
# Get country names which have more than 2 words
list_country_names = [re.sub('[,\"\n\'\(\)\.]', '', s) for s in open('country_names.txt', 'r').readlines()]

country_names_more_than_2_words = [s for s in list_country_names if len(s.split()) > 1]

In [0]:
# Create compound words
def insert_underscore(matchobj):
    return '' + matchobj.group(0).replace(' ', '_')

content = open('enwiki_tokenized.txt', 'r').read()

content = re.sub("(%s)" % "|".join(country_names_more_than_2_words), insert_underscore, content)

with open('enwiki_tokenized_compound_words.txt', 'w') as f:
    f.write(content)

In [0]:
# To display word has _
!grep -oh "\w*_\w*" enwiki_tokenized_compound_words.txt